In [1]:
import pandas as pd
import numpy as np
import re
from scipy import stats

## Import Cities Dataset

In [2]:
cities_df = pd.read_html('assets/wikipedia_data.html')[1]

In [82]:
#cities_df.head()

In [4]:
cities_df.rename({'Population (2016 est.)[8]': 'Population'}, axis=1, inplace=True)

In [5]:
cities_df = cities_df.iloc[:-1, :]

In [6]:
cities_df['Population'] = cities_df['Population'].astype(np.float)

In [7]:
#cities_df

## Import NHL Dataset

In [8]:
nhl_df = pd.read_csv('assets/nhl.csv')

In [9]:
nhl_df = nhl_df[nhl_df['year'] == 2018]

In [83]:
#nhl_df.head()

In [84]:
#cities_df.head()

In [85]:
#re.findall('[A-Z][a-z]+', cities_df['NHL'][12])

In [13]:
metro_nhl = cities_df[['Metropolitan area', 'NHL']]
metro_nhl['team_LName'] = metro_nhl['NHL'].apply(lambda x: re.findall('[A-Z][a-z]+', x))
metro_nhl = metro_nhl.explode(column='team_LName').dropna()

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
metro_nhl.drop('NHL', axis=1, inplace=True)

In [15]:
nhl_index = metro_nhl['Metropolitan area'].unique()

In [16]:
nhl_df['team_LName'] = nhl_df['team'].apply(lambda x: re.findall('.+\s([A-Z1-9]+[a-z]+)\*?', x)[0])

In [86]:
#nhl_df.head()

In [18]:
merged_nhl = metro_nhl.merge(nhl_df, on='team_LName')
merged_nhl['W'] = merged_nhl['W'].astype(np.float)
merged_nhl['L'] = merged_nhl['L'].astype(np.float)
merged_nhl['W/L%'] = merged_nhl['W'] / (merged_nhl['W'] + merged_nhl['L'])

In [19]:
grouped_nhl = merged_nhl.groupby('Metropolitan area').agg({'W/L%': 'mean'}).merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area')
grouped_nhl.set_index('Metropolitan area', inplace=True)
q1 = grouped_nhl.loc[nhl_index, :]

## Import NBA Dataset

In [20]:
nba_df = pd.read_csv('assets/nba.csv')
nba_df = nba_df[nba_df['year'] == 2018]

In [21]:
nba_df['W/L%'] = nba_df['W/L%'].astype(np.float)

In [87]:
#nba_df.head()

In [88]:
#cities_df.head()

In [24]:
metro_nba = cities_df[['Metropolitan area', 'NBA']]

In [25]:
metro_nba['team_LName'] = metro_nba['NBA'].apply(lambda x: re.findall('[A-Z1-9]+[a-z]+', x))
metro_nba = metro_nba.explode(column='team_LName').dropna()
metro_nba.drop('NBA', axis=1, inplace=True)

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
nba_index = metro_nba['Metropolitan area'].unique()

In [27]:
nba_df['team_LName'] = nba_df['team'].apply(lambda x: re.findall('.*\s([A-Z1-9]+[a-z]+)\*?\s\S', x)[0])

In [28]:
merged_nba = nba_df.merge(metro_nba, on='team_LName')
grouped_nba = merged_nba.groupby(by='Metropolitan area').agg({'W/L%': 'mean'})

In [29]:
q2 = grouped_nba.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')

In [30]:
q2 = q2.loc[nba_index, :]

## Import MLB Dataset

In [31]:
mlb_df = pd.read_csv('assets/mlb.csv')
mlb_df = mlb_df[mlb_df['year'] == 2018]

In [89]:
#mlb_df.head()

In [90]:
#cities_df.head()

In [34]:
metro_mlb = cities_df[['Metropolitan area', 'MLB']]
metro_mlb['team_LName'] = metro_mlb['MLB'].apply(lambda x: re.findall('[A-Z][a-z]+\s[A-Z][a-z]+|[A-Z1-9]+[a-z]+', x))
metro_mlb = metro_mlb.explode(column='team_LName').dropna().drop('MLB', axis=1)

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
#metro_mlb

In [36]:
mlb_index = metro_mlb['Metropolitan area'].unique()

In [92]:
#mlb_df

In [38]:
mlb_df['team_LName'] = mlb_df['team'].apply(lambda x: re.findall('.+\s([A-Z][a-z]+)', x)[0])

In [39]:
mlb_df.iloc[0, -1] = 'Red Sox'
mlb_df.iloc[8, -1] = 'White Sox'
mlb_df.iloc[3, -1] = 'Blue Jays'

In [93]:
#mlb_df

In [41]:
merged_mlb = mlb_df.merge(metro_mlb, on='team_LName')
grouped_mlb = merged_mlb.groupby(by='Metropolitan area').agg({'W-L%': 'mean'})

In [42]:
q3 = grouped_mlb.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')

In [43]:
q3 = q3.loc[mlb_index, :]

## Import NFL Dataset

In [44]:
nfl_df = pd.read_csv('assets/nfl.csv')
nfl_df = nfl_df[nfl_df['year'] == 2018]

In [94]:
#nfl_df.head()

In [46]:
nfl_df.drop([0, 5, 10, 15, 20, 25, 30, 35], axis=0, inplace=True)

In [47]:
nfl_df['W-L%'] = nfl_df['W-L%'].astype(np.float)

In [95]:
#cities_df

In [49]:
metro_nfl = cities_df[['Metropolitan area', 'NFL']]
metro_nfl['team_LName'] = metro_nfl['NFL'].apply(lambda x: re.findall('[A-Z1-9]+[a-z]+', x))
metro_nfl = metro_nfl.explode(column='team_LName').dropna().drop('NFL', axis=1)

C:\Users\Mann\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [50]:
nfl_index = metro_nfl['Metropolitan area'].unique()

In [96]:
#metro_nfl.head()

In [97]:
#nfl_df.head()

In [53]:
nfl_df['team_LName'] = nfl_df['team'].apply(lambda x: re.findall('.*\s([A-Z1-9]+[a-z]+)\S*', x)[0])

In [54]:
merged_nfl = nfl_df.merge(metro_nfl, on='team_LName')
grouped_nfl = merged_nfl.groupby(by='Metropolitan area').agg({'W-L%': 'mean'})

In [55]:
q4 = grouped_nfl.reset_index().merge(cities_df[['Metropolitan area', 'Population']], on='Metropolitan area').set_index('Metropolitan area')

In [56]:
q4 = q4.loc[nfl_index, :]

In [57]:
q1.shape

(28, 2)

In [58]:
q2.shape

(28, 2)

In [59]:
q3.shape

(26, 2)

In [60]:
q4.shape

(29, 2)

# P-Values

In [61]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [62]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [63]:
q1_nhl = q1.copy()
q2_nba = q2.copy()
q3_mlb = q3.copy()
q4_nfl = q4.copy()

In [64]:
nfl_nba = pd.merge(q4_nfl, q2_nba, right_index=True, left_index=True)

In [98]:
p_values.loc['NFL', 'NBA'] = stats.ttest_rel(nfl_nba.iloc[:, 0], nfl_nba.iloc[:, 2])[1]
p_values.loc['NBA', 'NFL'] = p_values.loc['NFL', 'NBA']

In [99]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


In [67]:
nfl_nhl = pd.merge(q1_nhl, q4_nfl, right_index=True, left_index=True)

In [68]:
p_values.loc['NHL', 'NFL'] = stats.ttest_rel(nfl_nhl.iloc[:, 0], nfl_nhl.iloc[:, 2])[1]
p_values.loc['NFL', 'NHL'] = p_values.loc['NHL', 'NFL']

In [69]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.079504,0.030318,NaN
NBA,0.079504,NaN,NaN,NaN
NHL,0.030318,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [70]:
nfl_mlb = pd.merge(q3_mlb, q4_nfl, right_index=True, left_index=True)

In [71]:
p_values.loc['NFL', 'MLB'] = stats.ttest_rel(nfl_mlb.iloc[:, 0], nfl_mlb.iloc[:, 2])[1]
p_values.loc['MLB', 'NFL'] = p_values.loc['NFL', 'MLB']

In [72]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.079504,0.030318,0.803459
NBA,0.079504,NaN,NaN,NaN
NHL,0.030318,NaN,NaN,NaN
MLB,0.803459,NaN,NaN,NaN


In [74]:
nba_nhl = pd.merge(q1_nhl, q2_nba, right_index=True, left_index=True)
p_values.loc['NBA', 'NHL'] = stats.ttest_rel(nba_nhl.iloc[:, 0], nba_nhl.iloc[:, 2])[1]
p_values.loc['NHL', 'NBA'] = p_values.loc['NBA', 'NHL']

In [75]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.079504,0.030318,0.803459
NBA,0.079504,NaN,0.022386,NaN
NHL,0.030318,0.022386,NaN,NaN
MLB,0.803459,NaN,NaN,NaN


In [77]:
nba_mlb = pd.merge(q2_nba, q3_mlb, left_index=True, right_index=True)
p_values.loc['NBA', 'MLB'] = stats.ttest_rel(nba_mlb.iloc[:, 0], nba_mlb.iloc[:, 2])[1]
p_values.loc['MLB', 'NBA'] = p_values.loc['NBA', 'MLB']

In [78]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.079504,0.030318,0.803459
NBA,0.079504,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,NaN
MLB,0.803459,0.949566,NaN,NaN


In [80]:
nhl_mlb = pd.merge(q1_nhl, q3_mlb, left_index=True, right_index=True)
p_values.loc['NHL', 'MLB'] = stats.ttest_rel(nhl_mlb.iloc[:, 0], nhl_mlb.iloc[:, 2])[1]
p_values.loc['MLB', 'NHL'] = p_values.loc['NHL', 'MLB']

In [81]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.079504,0.030318,0.803459
NBA,0.079504,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
